In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data={'+':np.array([[1,2],[3,2],[1,1.9],[2,2.9]]),
      "*":np.array([[4,5],[5,5.9],[7,5],[6,4.5]])}

In [3]:
for i in data:
    plt.scatter(data[i][:,0],data[i][:,1],marker=i,s=150)
plt.scatter(3,4.2,marker='x',s=150)

In [4]:
a=np.array([3,4])
b=np.array([7,4.9])

In [5]:
np.sqrt(sum((a-b)**2))

4.1000000000000005

In [6]:
np.linalg.norm(a-b)

4.1000000000000005

In [7]:
from scipy.spatial.distance import minkowski,euclidean

In [8]:
minkowski(a,b,2)

4.1000000000000005

In [9]:
euclidean(a,b)

4.1000000000000005

In [10]:
from collections import Counter
import pandas as pd
def knn(dt,prd):
    d=[]
    for key in dt:
        for points in dt[key]:
            d.append((euclidean(points,prd),key))
    v=sorted(d)[:3]
    v=[i[1] for i in v]
    #Counter(v).most_common(1)[0][0]
    x=dict(pd.Series(v).value_counts())
    return max(x,key=x.get)

knn(data,np.array([3,4.2]))

'+'

In [11]:
import pandas as pd

In [12]:
dt=pd.read_csv("bcancer.csv")
dt.drop('Unnamed: 0',axis=1,inplace=True)
dt=dt.replace('?','15')
dt['g']=dt['g'].astype(dtype='i')
x=dt.drop(['a','class'],axis=1)
y=dt['class']
from sklearn.model_selection import train_test_split
x_tr,x_ts,y_tr,y_ts=train_test_split(x,y,test_size=0.2)

In [13]:
x_tr.shape,y_tr.shape

((558, 9), (558,))

In [14]:
datas=dict.fromkeys(y_tr.unique())
for key in datas:
    datas[key]=[]
datas

{2: [], 4: []}

In [15]:
for i in range(x_tr.shape[0]):
    datas[y_tr.iloc[i]].append(x_tr.iloc[i].values)

In [16]:
datas

{2: [array([5, 1, 1, 2, 2, 2, 3, 1, 1], dtype=int64),
  array([2, 1, 1, 2, 2, 1, 1, 1, 1], dtype=int64),
  array([4, 4, 4, 4, 6, 5, 7, 3, 1], dtype=int64),
  array([3, 1, 1, 1, 2, 1, 2, 1, 1], dtype=int64),
  array([4, 1, 2, 1, 2, 1, 2, 1, 1], dtype=int64),
  array([2, 1, 1, 1, 2, 5, 1, 1, 1], dtype=int64),
  array([5, 1, 1, 1, 2, 2, 3, 3, 1], dtype=int64),
  array([1, 2, 1, 3, 2, 1, 2, 1, 1], dtype=int64),
  array([ 5,  1,  1,  1,  2, 15,  3,  1,  1], dtype=int64),
  array([1, 1, 1, 1, 2, 1, 3, 1, 1], dtype=int64),
  array([3, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
  array([1, 3, 1, 1, 2, 1, 2, 2, 1], dtype=int64),
  array([1, 1, 1, 1, 2, 3, 3, 1, 1], dtype=int64),
  array([3, 1, 2, 2, 2, 1, 1, 1, 1], dtype=int64),
  array([5, 2, 2, 4, 2, 4, 1, 1, 1], dtype=int64),
  array([6, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
  array([3, 1, 1, 1, 2, 5, 1, 1, 1], dtype=int64),
  array([1, 3, 1, 2, 2, 2, 5, 3, 2], dtype=int64),
  array([1, 1, 1, 1, 2, 1, 1, 1, 1], dtype=int64),
  array([1, 1, 1, 1

In [19]:
c=0
for i in range(x_ts.shape[0]):
    if knn(datas,x_ts.iloc[i])==y_ts.iloc[i]:
        c+=1
c/len(x_ts)

0.9785714285714285

In [20]:
from sklearn.metrics import confusion_matrix

In [25]:
prd=[knn(datas,x_ts.iloc[i]) for i in range(x_ts.shape[0]) ]
confusion_matrix(y_ts,prd)

array([[90,  1],
       [ 2, 47]], dtype=int64)

In [22]:
from sklearn.metrics import classification_report

In [24]:
prd=[knn(datas,x_ts.iloc[i]) for i in range(x_ts.shape[0])]
print(classification_report(y_ts,prd))

             precision    recall  f1-score   support

          2       0.98      0.99      0.98        91
          4       0.98      0.96      0.97        49

avg / total       0.98      0.98      0.98       140



In [27]:
import warnings
class Knn:
    def __init__(s,n=3):
        if n%2==0:
            warnings.warn("given n is even will converted to n-1")
            s.n=n-1
        else:
            s.n=n
    def fit(s,x_tr,y_tr):
        s.datas=dict.fromkeys(y_tr.unique())
        for key in s.datas:
            s.datas[key]=[]
        for i in range(x_tr.shape[0]):
            s.datas[y_tr.iloc[i]].append(x_tr.iloc[i].values)
    def predict(s,prd):
        d=[]
        for key in s.datas:
            for points in s.datas[key]:
                d.append((euclidean(points,prd),key))
        v=sorted(d)[:s.n]
        v=[i[1] for i in v]
        x=dict(pd.Series(v).value_counts())
        return max(x,key=x.get)
    def score(s,x_ts,y_ts):
        c=0
        for i in range(x_ts.shape[0]):
            if s.predict(x_ts.iloc[i])==y_ts.iloc[i]:
                c+=1
        return c/len(x_ts)  

In [28]:
alg=Knn(4)
alg.fit(x_tr,y_tr)
alg.score(x_ts,y_ts)

C:\Users\amit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: given n is even will converted to n-1
  """


0.9785714285714285